# Erik's run script for the $Z_2$ gauge theory with Fermions

**Authors:**
- Clement Charles
- Erik Gustafson
- Elizabeth Hardt
- Florian Herren
- Norman Hogan
- Henry Lamm
- Sara Starecheski
- Ruth Van der Water
- Mike Wagman

The following code brings together all the necessary algorithms together to the simulations for our production run.

In [1]:
#Standard modules
import sys
import numpy as np
import yaml
import pandas
#Qiskit modules
from qiskit import IBMQ,QuantumCircuit, transpile, QuantumRegister, execute
from qiskit.providers.ibmq import RunnerResult
from qiskit import Aer
from qiskit.providers import aer
from qiskit.providers.aer import noise
from qiskit.providers.ibmq import least_busy
# from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.transpiler.passes.scheduling.alignments import check_durations
from qiskit.visualization import plot_histogram
from qiskit_ibm_provider import IBMProvider

import datetime
#Project modules
sys.path.append('./modules/')
sys.path.append('./inputs/')
sys.path.append('./plots/')
import StatePrep_Manny as sprep_m
import DD_PassManager as dd_pm
import paulitwirlingpass as pts
from Z2analysis import nsites2nqubits
import Trotterization as Trot

# for plotting
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from IPython.display import Image
plt.rcParams["figure.autolayout"] = True
colors = list(mcolors.TABLEAU_COLORS.keys())
%matplotlib inline

#For testing reload modules after edits
from importlib import reload
%load_ext autoreload

# Get input parameters from `yaml` file

In [2]:
### get run parameters from YAML input file ###

stream = open('inputs/Z2_input_flo.yaml', 'r')
dict_in = yaml.safe_load(stream)
stream.close()

account_info = dict_in['account_info'] # dictionary of account information
backend_opts = dict_in['backend_opts']   # dictionary of machine/simulator choices
simulation_opts = dict_in['simulation_opts'] # dictionary of lattice-simulation parameters
runtime_opts = dict_in['runtime_opts'] # dictionary of runtime parameters

### transpilation parameters ###
initial_layout = simulation_opts['layout']['initial_layout']
### Lattice parameters ###
mass = simulation_opts['lattice_params']['mass']       #fermion mass in lattice-spacing units
nsites = simulation_opts['lattice_params']['nsites']   #must be even for staggered simulation
nqubits = nsites2nqubits(nsites)
print(nqubits, nsites)
### Time evolution ###
tf = simulation_opts['time_evolution']['tf']       # total time interval in lattice-spacing units
ntrotter = simulation_opts['time_evolution']['ntrotter'] # number of trotter steps
epsilon = tf/ntrotter                              # step size in lattice-spacing units

### Pauli Twirling ###
twirl = simulation_opts['pauli_twirling']['twirl']     # if True, apply Pauli twirling
ntwirls = simulation_opts['pauli_twirling']['ntwirls'] # number of Pauli twirls

### dynamic decoupling ###
dd_method = simulation_opts['dynamic_decoupling']['method'] # if not none apply a given dynamic decoupling method

### Richardson ??? ###
richardson_level = simulation_opts['richardson_level'] # number of CNOTs for Richardson smearing

3 2


# Initialize IBM Account

In [3]:
### initialize IBM Quantum account ###

provider = IBMProvider()
hub = account_info['hub']
group = account_info['group']
project = account_info['project']
print(hub+"/"+group+"/"+project)
provider = IBMProvider(instance=hub+"/"+group+"/"+project)

ibm-q/open/main


# Choose target backend for simulations to run on

In [4]:
def get_backend(provider, machine, nqubits, simulator, sim_noise):
    
    ## Run on IBMQ qasm simulator
    if (simulator):
        backend = provider.get_backend('ibmq_qasm_simulator')
        
        if (sim_noise != "None"):
            device_backend = provider.get_backend(sim_noise)
            noise_model = noise.NoiseModel.from_backend(device_backend)
            backend.set_options(noise_model=noise_model.to_dict())
            
        print(f'Running on ibmq_qasm_simulator with {sim_noise} noise_model\n') 
        return backend
    ## Run on specified quantum computer(s)
    else:
#         devices = provider.backends(filters=lambda x: not (x.name() == 'ibmq_casablanca' or x.name() == 'ibmq_armonk')
#                                     and nqubits <= x.configuration().n_qubits <= 7
#                                     and not x.configuration().simulator
#                                     and x.status().operational == True)
#         print(type(devices))
#         print(type(devices[0]))
        #print(f'Backends matching critera ares {devices}\n')
        #print(device_names)
        
        if (machine == 'least_busy'):
            machine = str(least_busy(devices))
            backend = provider.get_backend(machine)    
        
        elif (machine[0:4] == 'ibmq_'):
            backend = provider.get_backend(machine)   
            
        else:
            backend = provider.get_backend(str(machine))
            
        print(f'Running on machine {machine}\n')
        return backend
        

In [5]:

# machine
# print(backend_opts['machine'])
machine = backend_opts['machine']           # machine name or "least busy"
simulator = backend_opts['qasm_simulator']  # if True, use ibmq_qasm_simulator 
noise_model = backend_opts['noise_model']   # None or device noise model 
print(noise_model)
backend = get_backend(provider, machine, nqubits, simulator, noise_model)

None
Running on ibmq_qasm_simulator with None noise_model



# Assign Runtime Program (and inputs)

assigns values from `yaml` file

In [6]:
# number of shots per simulation
nshots = runtime_opts['nshots'] 

# Not really sure what this is needed for...
program_id = runtime_opts['program_id']

# correct for readout error using calibration matrix if True
measurement_error_mitigation = runtime_opts['readout_error_mitigation'] 

#save results for each shot if True
memory = runtime_opts['memory'] 

# Degree of layout optimization by transpiler
# 1 is least optimized; 3 is most optimized
optimization_level = runtime_opts['optimization_level'] 

In [7]:
nsites2nqubits(nsites), nsites

(3, 2)

In [8]:
which_stateprep = "Manny"

def build_circuit(nq, nsteps, epsilon, mass, init_state=None,
                  dynamic_decoupling=False, Twirling=False):
    
#     print(nq)
    if init_state is None:
        if nq == 3:
            # For now, set coupling to one and use lowest lying state
            qc = QuantumCircuit(nq, nq)
            if which_stateprep == "Manny":
                qc.append(sprep_m.state_circuit(mass,interaction=1,E_n=0), [i for i in range(nq)])
            #elif which_stateprep == "Elizabeth":
            #    theta, E = sprep_e.three_q_SP(mass,g=1,n=0)
            #    print("Picked state with E =", E)
            #    print("Mixing angle theta = ", theta)
            #    qc.append(sprep_e.prepare_states_nsites2(nq,theta), [i for i in range(nq)])
        elif nq == 7:
            print('not implemented')

        qc.append(Trot.trotter_evolution(nsites, epsilon, mass, nsteps,
                                richardson_level=1),
                   [i for i in range(nq)])
#         for i in range(4):
        qc.h([i for i in range(0, nq, 2)])
        qc.measure([i for i in range(nq)], [i for i in range(nq)])
        return qc
#         return transpile(qc.decompose(), basis_gates=['cx', 'rz', 'sx'])
    else:
        print('not implemented')
        

In [9]:
qc = build_circuit(nqubits, 1, epsilon, mass)
qc.draw()

/home/florian/Projects/Fermilab/z2_matter_qc/simulation_main/./modules/StatePrep_Manny.py:107: RuntimeWarning: invalid value encountered in double_scalars
  if (np.linalg.norm(q)/np.linalg.norm(A[i]) < 1e-6):
/home/florian/Projects/Fermilab/z2_matter_qc/simulation_main/./modules/StatePrep_Manny.py:111: RuntimeWarning: invalid value encountered in divide
  q = q / np.linalg.norm(q)


┌──────────────┐┌──────────────┐┌───┐   ┌─┐   
q_0: ┤0             ├┤0             ├┤ H ├───┤M├───
     │              ││              │└───┘┌─┐└╥┘   
q_1: ┤1 circuit-117 ├┤1 circuit-120 ├─────┤M├─╫────
     │              ││              │┌───┐└╥┘ ║ ┌─┐
q_2: ┤2             ├┤2             ├┤ H ├─╫──╫─┤M├
     └──────────────┘└──────────────┘└───┘ ║  ║ └╥┘
c: 3/══════════════════════════════════════╩══╩══╩═
                                           1  0  2

# assemble circuits to run

In [14]:
circuits_no_dd = []
circuits_w_dd = []
tvals = []
twirl = False
for i in range(1, ntrotter + 1):
    print(i)
    # with dynamic decoupling
    qc = build_circuit(nqubits, i, epsilon, mass)
    qc.draw()
    if twirl:
        qcs = pts.randomly_compile(qc, ncopy=ntwirls, backend=backend,
                                   initial_layout=initial_layout)
        qcs = transpile(qcs, basis_gates=['cx', 'rz', 'sx', 'sxdg'], optimization_level=optimization_level)
        qcs = dd_pm.DD_PassManager(qcs, DD_sequence='CPMG',
                                   providerstr=[hub, group, project],
                                   backendstr=machine)
        circuits_w_dd += qcs
    else:
        qcs = transpile(qc, basis_gates=['cx', 'rz', 'sx'], optimization_level=optimization_level,
                        initial_layout=initial_layout)
        qcs = dd_pm.DD_PassManager(qcs, DD_sequence='XY4',
                                   providerstr=[hub, group, project],
                                   backendstr=machine)
        circuits_w_dd.append(qcs)
    # add rescaling circuit
    qc = build_circuit(nqubits, i, 0, mass)
    if twirl:
        qcs = pts.randomly_compile(qc, ncopy=ntwirls, backend=backend,
                                   initial_layout=initial_layout)
        qcs = transpile(qcs, basis_gates=['cx', 'rz', 'sx', 'sxdg'], optimization_level=optimization_level)
        qcs = dd_pm.DD_PassManager(qcs, DD_sequence='CPMG',
                                   providerstr=[hub, group, project],
                                   backendstr=machine)
        circuits_w_dd += qcs
    else:
        qcs = transpile(qc, basis_gates=['cx', 'rz', 'sx'], optimization_level=optimization_level,
                        initial_layout=initial_layout)
        qcs = dd_pm.DD_PassManager(qcs, DD_sequence='XY4',
                                   providerstr=[hub, group, project],
                                   backendstr=machine)
        circuits_w_dd.append(qcs)
    
circuits = circuits_w_dd

1
Picked state with E = -0.31743821405397804


IBMQProviderError: 'No provider matches the specified criteria: hub = ibm-q, group = open, project = main'

In [13]:

from qiskit.transpiler import PassManager, InstructionDurations

In [31]:
len(circuits)

40

In [32]:
circuits[0].draw()

global phase: 3π/2
     ┌─────────────────┐   ┌────┐      ┌────┐  ┌──────────┐        ░         »
q_0: ┤ Delay(2304[dt]) ├───┤ √X ├──────┤ √X ├──┤ Rz(-0.3) ├────────░─────────»
     ├─────────────────┤   └────┘      └────┘  └──────────┘        ░         »
q_1: ┤ Delay(1248[dt]) ├─────────────────────────────────────────────────────»
     ├─────────────────┤┌──────────┐   ┌────┐   ┌────────┐         ░         »
q_2: ┤ Delay(2464[dt]) ├┤ Rz(0.15) ├───┤ √X ├───┤ Rz(-π) ├─────────░─────────»
     └┬────────────────┤└──────────┘   └────┘   └─┬───┬──┘         ░         »
q_3: ─┤ Delay(160[dt]) ├──────────────────────────┤ X ├──────────────────────»
     ┌┴────────────────┤┌─────────┐               └─┬─┘                      »
q_4: ┤ Delay(2208[dt]) ├┤ Rz(0.3) ├─────────────────┼────────────────────────»
     └───┬─────────┬───┘└──┬────┬─┘ ┌─────────┐     │      ┌────────────────┐»
q_5: ────┤ Rz(π/2) ├───────┤ √X ├───┤ Rz(π/2) ├─────■──────┤ Delay(640[dt]) ├»
     ┌───┴─────────┴───┐┌──┴────┴──┐└──┬────┬─┘ ┌────────┐ └────────────────┘»
q_6: ┤ Delay(2048[dt]) ├┤ Rz(0.15) ├───┤ √X ├───┤ Rz(-π) ├───────────────────»
     └─────────────────┘└──────────┘   └────┘   └────────┘                   »
c: 7/════════════════════════════════════════════════════════════════════════»
                                                                             »
«     ┌───┐ ┌───────┐ ┌────────────────┐       ┌───┐       ┌────────────────┐»
«q_0: ┤ X ├─┤ Rz(π) ├─┤ Delay(352[dt]) ├───────┤ X ├───────┤ Delay(384[dt]) ├»
«     ├───┤┌┴───────┴┐└────────────────┘       ├───┴┐      └────────────────┘»
«q_1: ┤ X ├┤ Rz(0.3) ├────────■────────────────┤ √X ├────────────────────────»
«     └─┬─┘└─────────┘      ┌─┴─┐              ├───┬┘          ┌───────┐     »
«q_2: ──┼───────────────────┤ X ├──────────────┤ X ├───────────┤ Rz(π) ├─────»
«       │     ┌───┐       ┌─┴───┴─┐     ┌──────┴───┴──────┐    └─┬───┬─┘     »
«q_3: ──■─────┤ X ├───────┤ Rz(π) ├─────┤ Delay(1280[dt]) ├──────┤ X ├───────»
«       ░     ├───┤       ├───────┤     └┬────────────────┤      ├───┤       »
«q_4: ──░─────┤ X ├───────┤ Rz(π) ├──────┤ Delay(256[dt]) ├──────┤ X ├───────»
«       ░     ├───┴┐      └─┬────┬┘      └──┬──────────┬──┘      └───┘       »
«q_5: ────────┤ √X ├────────┤ √X ├──────────┤ Rz(-0.3) ├───────────■─────────»
«       ░     └────┘        └────┘          └──────────┘         ┌─┴─┐       »
«q_6: ──░────────────────────────────────────────────────────────┤ X ├───────»
«       ░                                                        └───┘       »
«c: 7/═══════════════════════════════════════════════════════════════════════»
«                                                                            »
«            ┌───┐       ┌───────┐┌────────────────┐       ┌───┐       »
«q_0: ───────┤ X ├───────┤ Rz(π) ├┤ Delay(352[dt]) ├───────┤ X ├───────»
«            └───┘       └───────┘└────────────────┘       └───┘       »
«q_1: ─────────────────────────────────────────────────────────────────»
«      ┌────────────────┐  ┌───┐  ┌────────────────┐       ┌───┐       »
«q_2: ─┤ Delay(640[dt]) ├──┤ X ├──┤ Delay(640[dt]) ├───────┤ X ├───────»
«     ┌┴────────────────┤  ├───┤  └───┬───────┬────┘┌──────┴───┴──────┐»
«q_3: ┤ Delay(1312[dt]) ├──┤ X ├──────┤ Rz(π) ├─────┤ Delay(1280[dt]) ├»
«     └┬────────────────┤  ├───┤      ├───────┤     └┬────────────────┤»
«q_4: ─┤ Delay(288[dt]) ├──┤ X ├──────┤ Rz(π) ├──────┤ Delay(256[dt]) ├»
«      └─────┬────┬─────┘  └───┘      └───────┘      └────────────────┘»
«q_5: ───────┤ √X ├────────────────────────────────────────────────────»
«            ├───┬┘      ┌───────┐┌────────────────┐       ┌───┐       »
«q_6: ───────┤ X ├───────┤ Rz(π) ├┤ Delay(624[dt]) ├───────┤ X ├───────»
«            └───┘       └───────┘└────────────────┘       └───┘       »
«c: 7/═════════════════════════════════════════════════════════════════»
«                                                                      »
«     ┌────────────────┐         ░            ┌───┐   

In [33]:
264 / 16

16.5

In [34]:
circuits_readout = []
qc = QuantumCircuit(nqubits, nqubits)
qc.measure_all()
circuits_readout.append(qc)
qc = QuantumCircuit(nqubits, nqubits)
qc.x([i for i in range(nqubits)])
qc.measure_all()
circuits_readout.append(qc)
circuits_readout = transpile(circuits_readout, initial_layout=initial_layout)

In [35]:
circuits = circuits + circuits_readout

In [36]:
job1 = backend.run(circuits, job_name='dynamic decoupling no twirling', shots=nshots)

In [18]:
jobs = [job1]
dataframe  = pandas.DataFrame(columns=['jobid', 'machine', 'date', 'sim type', 'nt', 'pauli twirling', 'dynamic decoupling', 'epsilon', 'mass', 'counts bare'])
data_row = []
ntwirls = 1
for k in range(len(jobs)):
    for j in range(42):
        data_dict = {}
        data_dict['jobid'] = jobs[k].job_id()
        data_dict['machine'] = jobs[k].backend().name()
        data_dict['date'] = datetime.datetime.today()
        if (j // ntwirls) % 2 == 0:
            data_dict['sim type'] = 'evolution'
            data_dict['nt'] = j // (ntwirls * 2) + 1
            data_dict['pauli twirling'] = twirl
            data_dict['dynamic decoupling'] = True
            data_dict['epsilon'] = epsilon
            data_dict['mass'] = mass
            data_dict['counts bare'] = jobs[k].result().results[j].data.counts
#             data_dict['counts mitigated'] = get_quasi_probabilities(jobs[k].result().results[i].data.counts)
            data_row.append(data_dict)
        else:
            data_dict['sim type'] = 'rescaling'
            data_dict['nt'] = j // (ntwirls * 2) + 1
            data_dict['pauli twirling'] = twirl
            data_dict['dynamic decoupling'] = True
            data_dict['epsilon'] = epsilon
            data_dict['mass'] = mass
            data_dict['counts bare'] = jobs[k].result().results[j].data.counts
#             data_dict['counts mitigated'] = get_quasi_probabilities(jobs[k].result().results[i].data.counts)
            data_row.append(data_dict)
        
dataframe = pandas.DataFrame(data_row)

dataframe.to_csv(f'no_twirl_dd_simulation_production_run_on_{machine}_data={datetime.datetime.today().date()}_{nqubits}qubits_{jobs[0].job_id()}.csv')

# build the circuit jobs

In [14]:
job1 = backend.run(circuits[:300], job_name='dynamic decoupling steps 1 - 5', shots=nshots)

In [16]:
job2 = backend.run(circuits[300:600], job_name='dynamic decoupling steps 6 - 10', shots=nshots)

In [17]:
job3 = backend.run(circuits[600:900], job_name='dynamic decoupling steps 11 - 15', shots=nshots)

In [18]:
job4 = backend.run(circuits[900:], job_name='dynamic decoupling steps 16 - 20', shots=nshots)

In [19]:
job5 = backend.run(circuits_readout, job_name='readout correction circuits', shots=nshots)

In [15]:
job1.status()

<JobStatus.QUEUED: 'job is queued'>

In [12]:

import qiskit
from qiskit import QuantumRegister,ClassicalRegister,QuantumCircuit,execute
from qiskit.compiler import transpile
import numpy as np
from numpy import pi as pi
import random
from qiskit import IBMQ
from qiskit.circuit.library import XGate, RZGate
from qiskit.transpiler import PassManager, InstructionDurations
from qiskit.transpiler.passes import ALAPSchedule, DynamicalDecoupling

In [ ]:
calibration_matrix = np.array([1])
for j in range(nqubits):
    matrix = np.zeros((2, 2))
    for key in job5.result().get_counts(0):
        if key[j] == '0':
            matrix[0, 0] += job5.result().get_counts(0)[key]
        else:
            matrix[1, 0] += job5.result().get_counts(0)[key]
    for key in job5.result().get_counts(1):
        if key[j] == '0':
            matrix[0, 1] += job5.result().get_counts(1)[key]
        else:
            matrix[1, 1] += job5.result().get_counts(1)[key]
    matrix /= nshots
    matrix = np.linalg.inv(matrix)
    calibration_matrix = np.kron(matrix, calibration_matrix)
    

In [1]:
def get_quasi_probabilities(counts):
    quasi_probs = np.zeros(2 ** nqubits)
    raw_counts = np.zeros(2 ** nqubits)
    for key in counts.keys():
        raw_counts[int(key, 16)] += counts[key] / nshots
    quasi_probs = calibration_matrix @ raw_counts
    return quasi_probs / np.sum(quasi_probs)

# build the runtime-object

In [ ]:
jobs = [job1, job2, job3, job4]

In [ ]:
dataframe  = pandas.DataFrame(columns=['jobid', 'machine', 'date', 'sim type', 'nt', 'pauli twirling', 'dynamic decoupling', 'epsilon', 'mass', 'counts bare'])
data_row = []
for k in range(len(jobs)):
    for j in range(300):
        data_dict = {}
        data_dict['jobid'] = jobs[k].job_id()
        data_dict['machine'] = jobs[k].backend().name()
        data_dict['date'] = datetime.datetime.today()
        if (j // ntwirls) % 2 == 0:
            data_dict['sim type'] = 'evolution'
            data_dict['nt'] = j // (ntwirls * 2) + 1
            data_dict['pauli twirling'] = twirl
            data_dict['dynamic decoupling'] = True
            data_dict['epsilon'] = epsilon
            data_dict['mass'] = mass
            data_dict['counts bare'] = jobs[k].result().results[j].data.counts
#             data_dict['counts mitigated'] = get_quasi_probabilities(jobs[k].result().results[i].data.counts)
            data_row.append(data_dict)
        else:
            data_dict['sim type'] = 'rescaling'
            data_dict['nt'] = j // (ntwirls * 2) + 1
            data_dict['pauli twirling'] = twirl
            data_dict['dynamic decoupling'] = True
            data_dict['epsilon'] = epsilon
            data_dict['mass'] = mass
            data_dict['counts bare'] = jobs[k].result().results[j].data.counts
#             data_dict['counts mitigated'] = get_quasi_probabilities(jobs[k].result().results[i].data.counts)
            data_row.append(data_dict)
        
dataframe = pandas.DataFrame(data_row)

In [ ]:
dataframe.to_csv(f'simulation_production_run_on_{machine}_data={datetime.datetime.today().date()}_{nqubits}qubits_{jobs[0].job_id()}.csv')